# Алгоритм Бойера-Мура
Время работы: от $O(n*m)$ на хороших данных до $O(n/m)$ на плохих данных, 
где n - длина строки, m - длина подстроки

In [19]:
from collections.abc import Callable
from colorama import Fore, Style

# Служебная функция для тестирования других функций
def test_function(func: Callable[[str],int],test_values: list[str])->None:
    for i in test_values:
        try:
            result = func(*i[0])
            print (f'Результат функции {func.__name__} на {i[0]} равен {result}')
            assert result == i[1], Fore.RED+ f'Тест с {i[0]} не пройден! {result} != {i[1]}'+Style.RESET_ALL
        except AssertionError as err:
            print(err)
        else:
            print(Fore.GREEN+f'Тест с {i[0]} пройден!'+Style.RESET_ALL)

## Идея 1. Эвристика плохого символа 

In [20]:
# функция считает самую правую позицию каждого символа в шаблоне
def table_bad_symbols(s:str) -> dict:
    res = {}
    for pos,val in enumerate(s):
        res[val] = pos
    return res

print('Протестируем функцию по подсчету самой правой позиции каждого символа в строке "asdasfg"')
test_values_tbs = [
    [['asdasfg'],{'a':3,'s':4,'d':2,'f':5,'g':6}]
]

test_function(table_bad_symbols,test_values_tbs)

# функция ищет подстроку с помощью таблицы плохих символов
def find_bad_symbol(s:str,substring:str) -> int:
    i = 0
    n = len(s)
    m = len(substring)
    table = table_bad_symbols(s)
    # начинаем прикладывать подстроку к строке и сравнивать символы с конца подстроки
    while i<=n-m:
        # j - последний символ подстроки
        j = m-1
        # пока символы совпадают, идем назад
        while s[i+j] == substring[j]:
            j-=1
            # если j==0, то прошли всю подстроку
            if j<0:
                return i
        # случай 1 - самое правое вхождение текущего символа строки находится внутри подстроки,
        # тогда смещаем i вправо на него
        # случай 2 - самое правое вхождение текущего символа строки находится вне пределов
        # прикладываемой подстроки (левее её), тогда просто сдвигаем i на единицу
        i += max(j-table[s[i+j]],1)
    return -1

print('Протестируем поиск подстроки с помощью таблицы плохих символов')
test_values = [
    [['treasure','sure'],4],
    [['qwerty','u'],-1],
    [['qwerqwerty','qwerty'],4]
]

test_function(find_bad_symbol, test_values)



Протестируем функцию по подсчету самой правой позиции каждого символа в строке "asdasfg"
Результат функции table_bad_symbols на ['asdasfg'] равен {'a': 3, 's': 4, 'd': 2, 'f': 5, 'g': 6}
Тест с ['asdasfg'] пройден!
Протестируем поиск подстроки с помощью таблицы плохих символов
Результат функции find_bad_symbol на ['treasure', 'sure'] равен 4
Тест с ['treasure', 'sure'] пройден!
Результат функции find_bad_symbol на ['qwerty', 'u'] равен -1
Тест с ['qwerty', 'u'] пройден!
Результат функции find_bad_symbol на ['qwerqwerty', 'qwerty'] равен 4
Тест с ['qwerqwerty', 'qwerty'] пройден!


## Эвристика хорошего символа

In [21]:
# функция  для каждого суффикса считает его самую правую позицию в тексте
# Например: 
# колокол
# ----012
# абракадабра
# -------0127
def table_good_suffix(s:str) -> dict:
    m = len(s)
    # суффиксов m+1 - считаем так же нулевой пустой суффикс
    res = [-1]*(m+1)
    res[m] = 0
    # последний символ не интересует
    for x in range(0,m-1):
        # x - правый символ подстроки, которая равна суффиксу
        i = x
        j = m-1
        while i>=0 and s[i]==s[j]:
            # суффикс, начинающийся с J  позиций равен подстроке, начинающейся с i позиции
            res[j] = i
            i-=1
            j-=1
    return res

test_values_suf = [
    [['колокол'], [-1,-1,-1,-1,0,1,2,0]],
    [['абракадабра'],[-1,]*7+[0,1,2,7,0]]
]

print('Протестируем функцию построения таблицы хороших суффиксов ')
test_function(table_good_suffix,test_values_suf)

# функция ищет подстроку с помощью таблицы хороших суффиксов
def find_good_suffix(s : str,substring : str) -> int:
    n = len(s)
    m = len(substring)
    suf = table_good_suffix(s)
    # прикладываем подстроку к строке, сравнивая символы с конца
    i = n-1
    while i>0 and i<n:
        j = m-1
        while s[i]==substring[j]:
            i-=1
            j-=1
            # если прошли до конца подстроку и все совпало,то возвращаем начало ее входа в строку
            if j==0:
                return i
        # j+1 - последний символ из подстроки, который совпадал со строкой
        # либо длина подстроки, если несовпадение пошло сразу
        # suf[j+1] - позиция, в которой должен быть этот суффикс
        # на место j+1 символа должен встать суффикс
        i += j+1-suf[j+1]
    return -1

print('Протестируем поиск подстроки с помощью таблицы хороших символов')
test_function(find_good_suffix,test_values)

Протестируем функцию построения таблицы хороших суффиксов 
Результат функции table_good_suffix на ['колокол'] равен [-1, -1, -1, -1, 0, 1, 2, 0]
Тест с ['колокол'] пройден!
Результат функции table_good_suffix на ['абракадабра'] равен [-1, -1, -1, -1, -1, -1, -1, 0, 1, 2, 7, 0]
Тест с ['абракадабра'] пройден!
Протестируем поиск подстроки с помощью таблицы хороших символов
Результат функции find_good_suffix на ['treasure', 'sure'] равен 4
Тест с ['treasure', 'sure'] пройден!
Результат функции find_good_suffix на ['qwerty', 'u'] равен -1
Тест с ['qwerty', 'u'] пройден!
Результат функции find_good_suffix на ['qwerqwerty', 'qwerty'] равен 4
Тест с ['qwerqwerty', 'qwerty'] пройден!
